In [2]:
import os
import re
import fitz
from tqdm import tqdm
from datasets import Dataset, DatasetDict

In [ ]:
PDF_DIR = "./domain_pdf"
TXT_DIR = "./domain_txt"
DATASET_DIR = "./domain_dataset"

os.makedirs(TXT_DIR, exist_ok=True)
os.makedirs(DATASET_DIR, exist_ok=True)

In [ ]:
def pdf_to_txt(pdf_path: str) -> str:
    """
    Извлекает текст из PDF-файла, используя PyMuPDF (fitz).
    Возвращает сырой текст со всех страниц файла.
    """
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

In [ ]:
def clean_text(text: str) -> str:
    """
    Очищает входной текст от:
      - лишних переносов строк
      - URL-ссылок (http/https)
      - HTML-тегов (<...>)
      - ссылок-цитат вида [1], [2]
      - непечатаемых и управляющих символов
      - множественных пробелов
      - символов вне разрешённого набора
      - лишних пробелов перед знаками препинания
    Возвращает очищенный текст.
    """
    text = re.sub(r'\r\n', '\n', text)
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'\[\d+\]', '', text)
    text = re.sub(r'[\x00-\x1F\x7F-\x9F]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r"[^А-Яа-яёЁA-Za-z0-9.,:;!?()\-\'\"\s]", '', text)
    text = re.sub(r'\s+([.,:;!?])', r'\1', text)
    return text.strip()

In [ ]:
pdf_files = [f for f in os.listdir(PDF_DIR) if f.lower().endswith(".pdf")]

all_texts = []
total_chars_raw = 0
total_chars_clean = 0

for filename in tqdm(pdf_files, desc="Обработка PDF"):
    pdf_path = os.path.join(PDF_DIR, filename)
    txt_filename = os.path.splitext(filename)[0] + ".txt"
    txt_path = os.path.join(TXT_DIR, txt_filename)

    try:
        raw_text = pdf_to_txt(pdf_path)
        total_chars_raw += len(raw_text)

        cleaned = clean_text(raw_text)
        total_chars_clean += len(cleaned)

        with open(txt_path, "w", encoding="utf-8") as f:
            f.write(cleaned)

        if cleaned.strip():
            all_texts.append({"text": cleaned})
        else:
            print(f"Пустой текст после очистки для файла: {filename}")

    except Exception as e:
        print(f"Ошибка при обработке файла {filename}: {e}")

print("\n========== СТАТИСТИКА ==========")
print(f"Всего PDF-файлов: {len(pdf_files)}")
print(f"Общее кол-во символов ДО очистки: {total_chars_raw}")
print(f"Общее кол-во символов ПОСЛЕ очистки: {total_chars_clean}")
print(f"Общее кол-во записей для датасета: {len(all_texts)}")
print("================================\n")

In [ ]:
dataset = Dataset.from_list(all_texts)

split_dataset = dataset.train_test_split(test_size=0.1, seed=42)

ds = DatasetDict({
    "train": split_dataset["train"],
    "validation": split_dataset["test"]
})

ds

In [ ]:
ds.save_to_disk(DATASET_DIR)